In [ ]:
# API 정보 : https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15059093

In [ ]:
import requests
import pandas as pd

In [ ]:
code_list = [['99',  '파주'   ],
             ['108', '서울'   ],
             ['112', '인천'   ],
             ['232', '천안'   ],
             ['119', '수원'   ],
             ['133', '대전'   ],
             ['143', '대구'   ],
             ['251', '고창군' ],
             ['172', '고창'   ],
            #  ['253', '김해시' ],
            #  ['152', '울산'   ],
            #  ['155', '창원'   ],
            #  ['156', '광주'   ],
            #  ['159', '부산'   ],
            #  ['202', '양평'   ],
             ]

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import requests
from google.cloud import bigquery
import pandas_gbq
from dateutil.relativedelta import relativedelta



In [ ]:
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
token = '__token_values__'
start_date = '20230410'
end_date = '20230410'
# start_date = (datetime.today() - relativedelta(days=1)).strftime('%Y%m%d')
# end_date  = (datetime.today() - relativedelta(days=1)).strftime('%Y%m%d')

params ={'serviceKey' : token, 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'JSON', 'dataCd' : 'ASOS', 'dateCd' : 'DAY', 'startDt' : start_date, 'endDt' : end_date, 'stnIds' : '119' }

response = requests.get(url, params=params, timeout = 20)

In [ ]:
# response.json()['response']['header']['resultCode'] =='00'

In [ ]:
response.json()['response']['body']['items']['item'][0]

{'stnId': '119',
 'stnNm': '수원',
 'tm': '2023-04-10',
 'avgTa': '12.4',
 'minTa': '4.1',
 'minTaHrmt': '0451',
 'maxTa': '19.2',
 'maxTaHrmt': '1514',
 'mi10MaxRn': '',
 'mi10MaxRnHrmt': '',
 'hr1MaxRn': '',
 'hr1MaxRnHrmt': '',
 'sumRnDur': '',
 'sumRn': '',
 'maxInsWs': '9.4',
 'maxInsWsWd': '160',
 'maxInsWsHrmt': '1217',
 'maxWs': '5.7',
 'maxWsWd': '230',
 'maxWsHrmt': '1705',
 'avgWs': '2.3',
 'hr24SumRws': '2029',
 'maxWd': '140',
 'avgTd': '1.0',
 'minRhm': '30',
 'minRhmHrmt': '1624',
 'avgRhm': '49.5',
 'avgPv': '6.6',
 'avgPa': '1010.5',
 'maxPs': '1018.7',
 'maxPsHrmt': '0103',
 'minPs': '1012.3',
 'minPsHrmt': '2206',
 'avgPs': '1015.3',
 'ssDur': '12.9',
 'sumSsHr': '10.7',
 'hr1MaxIcsrHrmt': '1200',
 'hr1MaxIcsr': '3.1',
 'sumGsr': '22.46',
 'ddMefs': '',
 'ddMefsHrmt': '',
 'ddMes': '',
 'ddMesHrmt': '',
 'sumDpthFhsc': '',
 'avgTca': '1.0',
 'avgLmac': '0.3',
 'avgTs': '14.3',
 'minTg': '-1.3',
 'avgCm5Te': '11.8',
 'avgCm10Te': '11.4',
 'avgCm20Te': '11.0',
 'avgCm30T

In [ ]:
def API_data_get(code_list):
    df = pd.DataFrame()
    for code1 in code_list:
        params ={'serviceKey' : token, 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'JSON', 'dataCd' : 'ASOS', 'dateCd' : 'DAY', 'startDt' : start_date, 'endDt' : end_date, 'stnIds' : code1 }
        try: 
            response = requests.get(url, params=params, timeout = 20)
            df_temp = pd.DataFrame.from_dict([response.json()['response']['body']['items']['item'][0]])
            df = pd.concat([df, df_temp], ignore_index = True) 
        except:
            pass
    return df

In [ ]:
start_date, end_date  = '20230410', '20230410'

code_list = ['108', '112', '133', '143', '251', '119', '159']
df = API_data_get(code_list)
df = df[['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'maxTa' ]]
df['tm'] = pd.to_datetime(df['tm'], format='%Y-%m-%d')


In [ ]:
df['avgTa'] = df['avgTa'].astype(float)
df['minTa'] = df['minTa'].astype(float)
df['maxTa'] = df['maxTa'].astype(float)


In [ ]:
df

,stnId,stnNm,tm,avgTa,minTa,maxTa
0,108,서울,2023-04-10,13.2,6.2,19.1
1,112,인천,2023-04-10,12.4,7.7,16.2
2,133,대전,2023-04-10,12.6,2.5,21.3
3,143,대구,2023-04-10,14.5,5.2,23.7
4,251,고창군,2023-04-10,13.4,3.5,20.7
5,119,수원,2023-04-10,12.4,4.1,19.2
6,159,부산,2023-04-10,14.2,10.6,17.9


In [3]:
from datetime import datetime, timedelta
import pandas as pd
import requests
from google.cloud import bigquery
import pandas_gbq
from dateutil.relativedelta import relativedelta

# 저장 위치 설정
project = 'boxwood-**************'
client = bigquery.Client(project)
dataset_id = 'openAPI'
table_id = 'weather_daily'
table_name = dataset_id +'.' +table_id

url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
token = '__token_values__'
# start_date = (datetime.today() - relativedelta(days=1)).strftime('%Y%m%d')
# end_date  = (datetime.today() - relativedelta(days=1)).strftime('%Y%m%d')

start_date, end_date  = '20230430', '20230430'

def API_data_get(code_list):
    df = pd.DataFrame()
    for code1 in code_list:
        params ={'serviceKey' : token, 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'JSON', 'dataCd' : 'ASOS', 'dateCd' : 'DAY', 'startDt' : start_date, 'endDt' : end_date, 'stnIds' : code1 }
        try: 
            response = requests.get(url, params=params, timeout = 20)
            df_temp = pd.DataFrame.from_dict([response.json()['response']['body']['items']['item'][0]])
            df = pd.concat([df, df_temp], ignore_index = True) 
        except:
            pass
    return df

def get_data(t1):
    code_list = ['108', '112', '133', '143', '251', '119', '159']
    df = API_data_get(code_list)
    df = df[['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'maxTa' ]]
    df['tm'] = pd.to_datetime(df['tm'], format='%Y-%m-%d')
    df['avgTa'] = df['avgTa'].astype(float)
    df['minTa'] = df['minTa'].astype(float)
    df['maxTa'] = df['maxTa'].astype(float)
    # pandas_gbq.to_gbq(df, table_name, project_id=project, if_exists = 'append' )
    # return u'Hellow word'
    return df

In [4]:
get_data('')

,stnId,stnNm,tm,avgTa,minTa,maxTa
0,108,서울,2023-04-30,12.7,7.9,17.9
1,112,인천,2023-04-30,11.5,8.4,16.0
2,133,대전,2023-04-30,13.5,7.1,19.8
3,143,대구,2023-04-30,16.7,9.6,23.7
4,251,고창군,2023-04-30,13.2,4.9,20.5
5,119,수원,2023-04-30,12.8,7.1,18.7
6,159,부산,2023-04-30,15.1,10.1,18.8


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 62 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   stnId           7 non-null      object        
 1   stnNm           7 non-null      object        
 2   tm              7 non-null      datetime64[ns]
 3   avgTa           7 non-null      object        
 4   minTa           7 non-null      object        
 5   minTaHrmt       7 non-null      object        
 6   maxTa           7 non-null      object        
 7   maxTaHrmt       7 non-null      object        
 8   mi10MaxRn       7 non-null      object        
 9   mi10MaxRnHrmt   7 non-null      object        
 10  hr1MaxRn        7 non-null      object        
 11  hr1MaxRnHrmt    7 non-null      object        
 12  sumRnDur        7 non-null      object        
 13  sumRn           7 non-null      object        
 14  maxInsWs        7 non-null      object        
 15  maxInsWsWd